<a href="https://colab.research.google.com/github/choarauc/form_ch/blob/main/GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 

In [ ]:
link = 'https://raw.githubusercontent.com/insaid2018/Term-4/master/CaseStudy/mobile_train.csv'

In [ ]:
df = pd.read_csv(link, sep=',')

In [ ]:
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


| Variable     | Description                                                        |
|--------------|--------------------------------------------------------------------|
| battery_power| Energie totale qu'une batterie peut stocker en une fois, mesurée en mAh|
| blue         | A le bluetooth ou non                                              |
| clock_speed  | Vitesse à laquelle le microprocesseur exécute les instructions       |
| dual_sim     | Supporte ou non la double simulation                               |
| fc           | Caméra avant méga pixels                                           |
| four_g       | A la 4G ou non                                                     |
| int_memory   | Mémoire interne en Gigaoctets                                      |
| m_dep        | Epaisseur mobile en cm                                             |
| mobile_wt    | Poids du téléphone mobile                                          |
| n_cores      | Nombre de cœurs du processeur                                       |
| pc           | Caméra primaire méga pixels                                        |
| px_height    | Hauteur de la résolution en pixels                                  |
| px_width     | Largeur de la résolution en pixels                                  |
| ram          | RAM en méga octets                                                 |
| sc_h         | Hauteur de l'écran du mobile en cm                                  |
| sc_w         | Largeur de l'écran du mobile en cm                                  |
| talk_time    | Durée maximale de la batterie en communication                     |
| three_g      | A la 3G ou non                                                     |
| touch_screen | A un écran tactile ou non                                          |
| wifi         | A le wifi ou non                                                   |
| price_range  | C'est la variable cible avec les valeurs 0 (faible coût), 1 (coût moyen), 2 (coût élevé) et 3 (coût très élevé).|


+ Les variables sont toutes numériques, bien que certaines soient des variables indicatrices.

+ Toutes les colonnes ont l'air correctement distribué, à priori aucune ligne n'a de valeur aberrante.

> Nous allons choisir d'entraîner une régression logistique
>
>> il sera nécessaire de modifier l'échelle des données, qui sont a priori assez différentes.

+ Une fois la méthode d'apprentissage choisie, la sélection de modèle peut se résumer au choix des meilleurs hyperparamètres.
Dans le domaine de l'**Apprentissage Supervisé**, les "hyperparamètres" sont les paramètres de la méthode d'apprentissage elle-même que nous devons spécifier a priori, c'est-à-dire avant l'entraînement du modèle.

# 1. Sélection d'hyperparamètres

> Pour chaque ensemble d'hyperparamètres, nous calculons le score moyen et sélectionnons donc l'ensemble d'hyperparamètres ayant la meilleure performance moyenne sur l'ensemble des échantillons.

Cette méthode est réalisable grâce à **GridSearchCV**

Ensuite, le modèle est réentraîné avec les hyperparamètres sélectionnés sur l'ensemble d'entraînement complet

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

+ Séparer dans deux DataFrames les variables explicatives et la variable cible.
+ Créer un ensemble d'entraînement et un ensemble de tests, représentants respectivement 75% et 25% des données originales.
+ Normaliser les données explicatives dans les deux ensembles, grâce à une transformation Min-Max calculée sur les données d'entraînement.

In [ ]:
X, y = df.drop('price_range', axis=1), df['price_range']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

scaler = MinMaxScaler()

# Le MinMaxScaler est une technique de mise à l'échelle des données qui réajuste les valeurs d'une variable numérique dans un intervalle spécifié (les mettre à l'échelle), 
# généralement entre 0 et 1, en préservant les proportions relatives des données d'origine.

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

+ Créer un classifieur clf_lr, de la classe LogisticRegression, avec le paramètre max_iter=1000.
+ Créer un dictionnaire params_lr avec les hyperparamètres 'solver': ['liblinear', 'lbfgs'], et 'C': [10**(i) for i in range(-4, 3)].
+ Instancier une classe gridcv de GridSearchCV, avec clf_lr, les paramètres de params_lr.

In [ ]:
clf_lr = LogisticRegression(max_iter=1000)

# Définition d'une grille de paramètres à explorer lors de la recherche
params_lr = {'solver': ['liblinear', 'lbfgs'], 'C': [10**(i) for i in range(-4, 3)]}

# Création d'un objet GridSearchCV pour effectuer une recherche sur la grille des paramètres
# Utilisation de l accuracy pour évaluer les performances du modèle
# Utilisation d'une validation croisée à 3 plis (3-fold cross-validation)

gridcv = GridSearchCV(clf_lr, param_grid=params_lr, scoring='accuracy', cv=3)    

In [ ]:
# Ne pas lancer, ce code permet d'explorer tous les paramètres 

from sklearn.model_selection import ParameterGrid

clf_lr = LogisticRegression(max_iter=1000)

params_lr = {'solver': ['liblinear', 'lbfgs'], 'C': [10**(i) for i in range(-4, 3)]}

param_grid = list(ParameterGrid(params_lr))

# Boucle sur tous les paramètres pour effectuer la recherche
for params in param_grid:
    clf_lr.set_params(**params)

+ Entraîner à présent gridcv sur l'ensemble d'entraînement.
+ Afficher les résultats du GridSearch grâce à l'attribut cv_results_, dont on affichera les colonnes 'params', 'mean_test_score' et 'std_test_score'.

In [ ]:
gridcv.fit(X_train, y_train)

pd.DataFrame(gridcv.cv_results_)[['params', 'mean_test_score', 'std_test_score']]

,params,mean_test_score,std_test_score
0,"{'C': 0.0001, 'solver': 'liblinear'}",0.258667,0.000943
1,"{'C': 0.0001, 'solver': 'lbfgs'}",0.258667,0.000943
2,"{'C': 0.001, 'solver': 'liblinear'}",0.322667,0.029044
3,"{'C': 0.001, 'solver': 'lbfgs'}",0.512667,0.000943
4,"{'C': 0.01, 'solver': 'liblinear'}",0.536000,0.005888
5,"{'C': 0.01, 'solver': 'lbfgs'}",0.580667,0.007717
6,"{'C': 0.1, 'solver': 'liblinear'}",0.636000,0.020067
7,"{'C': 0.1, 'solver': 'lbfgs'}",0.726667,0.011813
8,"{'C': 1, 'solver': 'liblinear'}",0.734000,0.018403
9,"{'C': 1, 'solver': 'lbfgs'}",0.864667,0.019956


Les résultats affichés peuvent nous permettre de choisir l'ensemble d'hyperparamètres ayant obtenu le meilleur score :
    
    Il s'agit de {'C': 100, 'solver': 'lbfgs'}.

+ nous extrayons les meilleurs paramètres trouvés par GridSearchCV à partir de l'attribut best_params_. Ensuite, nous créons un nouveau modèle clf_lr_best de LogisticRegression en utilisant les meilleurs paramètres trouvés.

+ Nous entraînons ensuite le modèle clf_lr_best avec les données d'entraînement originales (non mis à l'échelle). Ensuite, nous utilisons ce modèle pour faire des prédictions sur les données de test X_test, et nous évaluons les performances du modèle en calculant l'exactitude (accuracy) en comparant les prédictions avec les vérités terrain y_test.

+ Enfin, nous affichons l'exactitude (accuracy) pour évaluer les performances du modèle sur les données de test.

In [ ]:
# Extraction des meilleurs paramètres trouvés par GridSearchCV
best_params = gridcv.best_params_

# Création d'un nouveau LogisticRegression avec les meilleurs paramètres
clf_lr_best = LogisticRegression(max_iter=1000, solver=best_params['solver'], C=best_params['C'])

# Entraînement du modèle avec les données d'entraînement originales (non mis à l'échelle)
clf_lr_best.fit(X_train, y_train)

# Prédiction sur les données de test
y_pred = clf_lr_best.predict(X_test)

# Évaluation des performances du modèle sur le test
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.95


In [ ]:
# prédictions sur les données de test
y_pred = clf_lr_best.predict(X_test)

df_predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# df des prédictions
df_predictions


,Actual,Predicted
396,3,3
1308,3,3
57,3,3
1112,3,3
310,3,3
...,...,...
329,1,1
611,1,1
222,3,3
1843,3,3


In [ ]:
# Filtrer sur les prédictions incorrectes
incorrect_predictions = df_predictions[df_predictions['Actual'] != df_predictions['Predicted']]

incorrect_predictions